In [12]:
import requests
from bs4 import BeautifulSoup

def scrape_site(url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        header = soup.find(['h1']).get_text().strip()
        content_tags = soup.find_all(['p'])
        content = [tag.get_text().strip().replace('\xa0', ' ') for tag in content_tags]
        return header, content
    else:
        print(f"Failed to retrieve the webpage. Status code: {response.status_code}")

url = "https://www.cnn.com/2024/01/17/politics/biden-ukraine-white-house-meeting/index.html"
header, content = scrape_site(url)

In [5]:
import context_veracity
content_concat = " ".join(content)
sent_score = context_veracity.sentiment_shift(content_concat)
topic_score = context_veracity.topic_shift(content_concat)
ner_score = context_veracity.ner_shift(content_concat)
context_veracity_score = context_veracity.calculate_contextual_drift(sent_score, topic_score, ner_score)
print(f"The context veracity score of the article is {context_veracity_score}")

/home/zhj003/.local/lib/python3.9/site-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


The context veracity score of the article is 4.3


In [8]:
import political_bias
import numpy as np
pred_labels = []
for paragraph in content:
    processed_article = political_bias.preprocess_article(header, paragraph)
    label = political_bias.predict_label(processed_article)
    pred_labels.append(label[0])
print("The political bias score of the article is ", np.mean(pred_labels))

The political bias score of the article is  1.0


In [25]:
from sentiment import SentimentModel
with open('models/sentiM.pkl', 'rb') as f:
            sentM = pickle.load(f)
pred_labels = []
for paragraph in content:
    label = sentM.predict_article(header, paragraph)[0]
    pred_labels.append(label)
print("The sentiment score of the article is ", np.mean(pred_labels))

The sentiment score of the article is  0.0
